# Save points and matrix profile

This file calculates and saves the data as pickle file, for every point belongs to a chosen water authority.

**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
from kando import kando_client
from tqdm import tqdm
import pickle
import os.path
from os import path
import datetime
%run utils.ipynb

**Global Variables**

In [2]:
RESAMPLE = '5min'
WINDOW = 24
WATER_AUTHORITY_NUM = '24'
START = datetime.datetime(2020, 1, 1, 0, 0).timestamp()

In [5]:
def create_df_from_dict(site_dic):
    """
    :param df: dictionary with raw data from kando's API
    :return: raw data in pandas dataframe with 'EC', 'PH', 'ORP', 'TEMPERATURE' columns,
             date is the index
    """
        
    df = pd.DataFrame.from_dict(site_dic['samplings'], orient='index')
    df['Date'] = df['DateTime'].copy()
    df['DateTime'] = pd.to_datetime(df['DateTime'], unit='s')
    df = df.set_index('DateTime')
    df = df[['Date', 'EC', 'PH', 'ORP', 'TEMPERATURE']]
    return(df)

In [4]:
def replace_outliers_with_medean(df,
                                 ecmin=100,
                                 ecmax=110000,
                                 phmin=0,
                                 phmax=14,
                                 orpmin=-400,
                                 orpmax=400,
                                 tempmin=5,
                                 tempmax=100):
    """
    :param df: raw data in pandas dataframe
    :param ecmin: min value for EC, less than that will considered an outlier
    :param ecmax: max value for EC, more than that will considered an outlier
    :param phmin: min value for PH, less than that will considered an outlier
    :param phmax: max value for PH, more than that will considered an outlier
    :param orpmin: min value for ORP, less than that will considered an outlier
    :param orpmax: max value for ORP, more than that will considered an outlier
    :param tempmin: min value for TEMPERATURE, less than that will considered an outlier
    :param tempmax: max value for TEMPERATURE, more than that will considered an outlier

    :return: data frame with all nulls and outliers imputed with median
    """
    
    safs = {
        'EC': (ecmin, ecmax),
        'PH': (phmin, phmax),
        'ORP': (orpmin, orpmax),
        'TEMPERATURE': (tempmin, tempmax)
    }
    for col in ['EC', 'PH', 'ORP', 'TEMPERATURE']:
        safs_range = (df[col] > safs[col][0]) & (df[col] < safs[col][1])
        outliers_range = ~safs_range
        median = df.loc[safs_range, col].median()
        df.loc[outliers_range, col] = np.nan
        df.fillna(median, inplace=True)
    return(df)

In [6]:
def filter_year(df, year):
     """
    :param df: data frame with sensorial data and datetime index
    :param year: requested year
    :return: same data frame filtered to the requested year
    """
        
    df['Date'] = pd.to_datetime(df['Date'], unit='s')
    df['year'] = df['Date'].apply(lambda x: x.year)
    df = df.drop(['Date'], axis=1)
    df = df[df.year==year]
    return(df)

In [7]:
def save_df(df, point_id):
     """
    :param df: data frame desired to be saved as a file
    :param point_id: id number of the point that the data belongs to
    :return: save the data frame as a pickle file
    """
        
    b_file = open(f"{point_id}_data.pkl", "wb")
    pickle.dump(df, b_file)
    b_file.close()

In [8]:
def save_matrix_profile_dictionary(mps, point_id):
     """
    :param df: data frame desired to be saved as a file
    :param point_id: id number of the point that the data belongs to
    :return: save the natrix profile calculated dictionary as a pickle file
    """
        
    a_file = open(f"{point_id}_mps.pkl", "wb")
    pickle.dump(mps, a_file)
    a_file.close()

In [3]:
with open('key.json') as f:
    api_login = json.load(f)

url = "https://kando-staging.herokuapp.com"
client = kando_client.client(url, api_login['key'], api_login['secret'])

In [9]:
with open('water_authority.json') as f:
    active_by_aouthority = json.load(f)

In [10]:
aouthority = active_by_aouthority[WATER_AUTHORITY_NUM]['nodes']

In [11]:
failed = []
for point_id in tqdm(aouthority):
    if not path.exists(f'{point_id}_mps.pkl'):
        try:
            site_dic = client.get_all(point_id=point_id, start = START)
            df = create_df_from_dict(site_dic)
            df = impute_nulls_with_time_interpolation(df, df.columns, RESAMPLE)
            df = replace_outliers_with_medean(df)
            df = filter_year(df, 2020)
            mps = create_matrix_profile_dictionary(df, ['EC', 'PH', 'ORP', 'TEMPERATURE'], m=WINDOW)
            save_df(df, point_id)
            save_matrix_profile_dictionary(mps, point_id)
            print(f'{point_id} mps was successfully saved')
        except:
            print(f'failed to create {point_id} mps')
            failed.append(point_id)
    else:
        print(point_id, 'files already exist in working directory')
print('Failed to create and save matrix profiles for these points:\n', failed)

Kando - GET /api/data/fetch?point_id=882&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=882&unit_id=&start=1593928800&end=&raw_data=
failed to create 882 mps
Kando - GET /api/data/fetch?point_id=883&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=883&unit_id=&start=1593928800&end=&raw_data=



883 mps was successfully saved
Kando - GET /api/data/fetch?point_id=888&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=888&unit_id=&start=1585839600&end=&raw_data=
Kando - GET /api/data/fetch?point_id=888&unit_id=&start=1593918000&end=&raw_data=



888 mps was successfully saved
Kando - GET /api/data/fetch?point_id=891&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=891&unit_id=&start=1585219800&end=&raw_data=
Kando - GET /api/data/fetch?point_id=891&unit_id=&start=1593918000&end=&raw_data=



891 mps was successfully saved
Kando - GET /api/data/fetch?point_id=896&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=896&unit_id=&start=1585316700&end=&raw_data=
Kando - GET /api/data/fetch?point_id=896&unit_id=&start=1592809500&end=&raw_data=
Kando - GET /api/data/fetch?point_id=896&unit_id=&start=1593918000&end=&raw_data=



896 mps was successfully saved
Kando - GET /api/data/fetch?point_id=903&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=903&unit_id=&start=1585313700&end=&raw_data=
Kando - GET /api/data/fetch?point_id=903&unit_id=&start=1592793300&end=&raw_data=
Kando - GET /api/data/fetch?point_id=903&unit_id=&start=1593918000&end=&raw_data=


...


4067 mps was successfully saved
Kando - GET /api/data/fetch?point_id=4210&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=4210&unit_id=&start=1593928800&end=&raw_data=



4210 mps was successfully saved
Kando - GET /api/data/fetch?point_id=4338&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=4338&unit_id=&start=1592942400&end=&raw_data=
Kando - GET /api/data/fetch?point_id=4338&unit_id=&start=1593918000&end=&raw_data=
failed to create 4338 mps
Kando - GET /api/data/fetch?point_id=4386&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=4386&unit_id=&start=1593928800&end=&raw_data=



4386 mps was successfully saved
Kando - GET /api/data/fetch?point_id=5056&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=5056&unit_id=&start=1578974400&end=&raw_data=



5056 mps was successfully saved
Kando - GET /api/data/fetch?point_id=5058&unit_id=&start=1577829600.0&end=&raw_data=
Kando - GET /api/data/fetch?point_id=5058&unit_id=&start=1588852800&end=&raw_data=
Kando - GET /api/data/fetch?point_id=5058&unit_id=&start=1593918000&end=&raw_data=



5058 mps was successfully saved

Failed to create and save matrix profiles for these points:
 [882, 985, 1994, 2292, 2294, 2295, 3226, 3668, 4338]
